# Blatt 9

## Aufgabe 8.3

In [1]:
import random, math
def monteCarlo(runAmount = 900):
    m = 1000
    resultInside = 0
    
    for i in range(0, runAmount):
        if(math.sqrt(2 * (float(random.randint(0, m) / m) * float(random.randint(0, m) / m)))) <= 1:
            resultInside += 1
            
    return (resultInside / runAmount) * 4

In [2]:
import multiprocessing
import itertools
import random, math
def monteCarloParallelMap(runAmount = 900):
    cores = multiprocessing.cpu_count()
    l = []
    
    for i in range(0, runAmount):
        l.append((i, 0))
        
    process_pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    result = process_pool.starmap(mapLoop, l)
    resultInside = sum(result)
    process_pool.close()
    
    return (resultInside / runAmount) * 4
def mapLoop(act, isInside):
    m = 1000
    
    if(math.sqrt(2 * (float(random.randint(0, m) / m) * float(random.randint(0, m) / m)))) <= 1:
        isInside = 1
        
    return isInside

In [3]:
import multiprocessing
import itertools
import random, math
def monteCarloParallelQueue(runAmount = 900):
    cores = multiprocessing.cpu_count()
    argumentQueue = multiprocessing.JoinableQueue()
    resultQueue = multiprocessing.Queue()
    processes = [
        multiprocessing.Process(
            target = queueLoop,
            args = (argumentQueue, resultQueue)
        ) for i in range(multiprocessing.cpu_count())
    ]
    
    for i in range(0, runAmount):
        argumentQueue.put(i)
        
    for p in processes:
        p.start()
        
    argumentQueue.join()
    
    for p in processes:
        p.terminate()
    
    resultInside = 0
    
    for i in range(0, runAmount):
        resultInside += int(resultQueue.get())
        
    return str((resultInside / runAmount) * 4)
def queueLoop(inQ, outQ):
    m = 1000
    
    while True:
        inQ.get()
        result = 0
        
        if(math.sqrt(2 * (float(random.randint(0, m) / m) * float(random.randint(0, m) / m)))) <= 1:
            result = 1
            
        outQ.put(result)
        inQ.task_done()

In [ ]:
totalRuns = 900

%timeit monteCarlo(totalRuns)
%timeit monteCarloParallelMap(totalRuns)
%timeit monteCarloParallelQueue(totalRuns)

print(monteCarlo(totalRuns))
print(monteCarloParallelMap(totalRuns))
print(monteCarloParallelQueue(totalRuns))

## Aufgabe 8.4

In [21]:
import numpy
nrOfListEntries = 10**6
a = numpy.random.normal(3, 1, size = nrOfListEntries).astype(numpy.float32)
#a = numpy.random.normal(3, 1, size = nrOfListEntries).astype(numpy.float64)

In [22]:
print(a)

[2.476603  2.6687083 1.5538356 ... 2.2742844 3.363559  2.733765 ]


In [23]:
%load_ext Cython
#%reload_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [24]:
%%cython -f -c-fopenmp --link-args=-fopenmp


"""
Implementation of mean and standard deviation calculation using
cython.parallel
"""
#from __future__ import with_statement
from cython.parallel import prange
import numpy
cimport numpy
cimport cython
from libc.math cimport sqrt

ctypedef numpy.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef numpy.ndarray[dtype_t, ndim=1] cython_stat_parallel(
        numpy.ndarray[dtype_t, ndim=1] l):
    """
    calculate mean and standard deviation of data stored in a list
    using cython parallel.
    Args:
        l numpy array containing numbers
    Returns:
        list [mean, standardDeviation] 
    """
    cdef double average, standard_deviation, skewness, kurtosis, tmp
    cdef double accumulator = 0.0
    cdef long N, i

    N = len(l)
    for i in prange(N, nogil=True):
        accumulator += l[i]
    average = accumulator / N
    accumulator = 0.0
    for i in prange(N, nogil=True):
        tmp = l[i] - average
        accumulator += tmp * tmp
        skewness += tmp * tmp * tmp
        kurtosis += tmp * tmp * tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))
    skewness = skewness / pow(standard_deviation, 3)
    kurtosis = kurtosis / pow(standard_deviation, 4)
    
    result = numpy.array((average, standard_deviation, skewness, kurtosis), numpy.float32)
    return result

In [30]:
#%timeit cython_stat_parallel(a)
print(cython_stat_parallel(a))

[3.0022352e+00 9.9983335e-01 8.7802014e+02 3.0011998e+06]
